# SQLITE with Python

#### What is SQL?
SQL(Structured Query Language) is a language used to manage data in relation databases. Think of it like Excel formulas but for databases. It lets you create, read, update and delete data in a structured way. Standardized in 1986, there are several different dialects.

#### SQL Terms
Table: Data structure which organises data into rows and columns.

Column: A field in the table which stores particular attributes for records(price, name, symbol). Each column can hold a specific data type.

Row: A single record containing values for each column in the table.

Query: A request to retrieve, insert, update or delete data from the database
#### SQL Data Types
INTEGER: Whole numbers(positive, negative, zero), can also be used for IDs and auto increament primary keys. Usage example coin ID INTEGER PRIMARY KEY

REAL: Floating point numbers(decimals). Good for prices, percentages etc

TEXT: Any string of characters. For names,symbols,descriptions etc i.e name TEXT, symbol TEXT

DATE/TEXT: SQlite has no dedicated DATE TYPE-dates are usually stored as TEXT in yyyy-MM-DD format, or as an INTEGER(Unix timestamp) i.e date TEXT.

#### SQL Commands
SELECT: We use the SELECT command to read data, probably the most common command for data Analytics(Dune queries read Dune's DB W/SELECT command access)

Insert-is used to add record DB management

Update-used to modify existing data

Delete-used to remove records

#### SQL Functions
SUM: calculates total of a column(SELECT SUM(volume)) from dex trades.

MAX: returns max volume in a column(SELECT MAX(price) from prices).

MIN: returns min  value in a column(SELECT MIN(tx_count)) from daily_tx.

AVG: calculates the average  of a column(SELECT AVG(users)) from weekly users.

COUNT:counts the number of rows in a column(SLECT COUNT(addrees)) from ledger.

REAL world example

Most blockchain data analysts exposed to SQL through Dune Analytics.

Dune allows users to query blockchain data using SQL queries on Dune's DB.

Dune collects transaction and event data for several blockchains and allows users to create  dashboards and download the data through APIs

##### Core SQL Commands
Here are the four essential SQL commands(often called CRUD operations)

SELECT-READ data

SELECT*FROM coins: gets all data from the coins table

SELECT name, price FROM coins WHERE price>100: Gets specific columns with a condition

INSERT :add new records

INSERT INTO coins VALUES('Bitcoin', 50000): Adds a new row

INSERT INTO coins(names,prices) VALUES('Ethereum' , 3000): Adds to specific columns

UPDATE-Modify existing data i.e UPDATE coins SET price-51000 WHERE name-'Bitcoin': Changes bitcoins price

DELETE-Remove records

DELETE FROM coins WHERE price<10> Deletes all cheap coins


Setting up a Dtabase. Lets create a simple database to track cryptocurrencies

In [2]:
import sqlite3
import pandas as pd

In [3]:
import os
db_path="Crypto.db"
if os.path.exists(db_path):
    os.remove(db_path)
    print("Database file deleted")
else:
    print("Database file does not exist.")

Database file deleted


In [41]:
# Create or connect database file
conn=sqlite3.connect(db_path)
# Create a cursor object to execute SQL commands
cursor=conn.cursor()
# Create a table
cursor.execute("""
               CREATE TABLE IF NOT EXISTS coins(
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               name TEXT NOT NULL,
               symbol text not null,
               price REAL,
               market_cap REAL,
               last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
               )
               """)
# Save() the changes
conn.commit
print("Table 'coins'created successfully.")

Table 'coins'created successfully.


## Inserting Data

In [42]:
# inserting data( insert a single record)
cursor.execute("""
               INSERT INTO coins(name,symbol,price,market_cap)
               VALUES('Bitcoin', 'BTC', '50000', 950000000000)
               """)

# Insert multiple records
coins=[
    ('Ethereum', 'ETH',3000, 3500000000),
    ('Cadano', 'ADA', 1.5, 50000000000),
    ('Solana', 'SOL', 150, 50000000000)
]
cursor.executemany("""
INSERT INTO coins(name,symbol,price, market_cap)
                    VALUES(?,?,?,?)
                    """, coins)
conn.commit()

## Querying Data

In [43]:
# Get all coins
cursor.execute("SELECT*FROM coins")
all_coins=cursor.fetchall() # Get all results as a list of turples.
print("All coins:", all_coins)

All coins: [(1, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:07:32'), (2, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:19:48'), (3, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:21:13'), (4, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:22:28'), (5, 'Ethereum', 'ETH', 3000.0, 3500000000.0, '2025-08-18 13:22:28'), (7, 'Solana', 'SOL', 150.0, 50000000000.0, '2025-08-18 13:22:28'), (8, 'Bitcoin', 'BTC', 50000.0, 950000000000.0, '2025-08-18 17:18:50'), (9, 'Ethereum', 'ETH', 3000.0, 3500000000.0, '2025-08-18 17:18:50'), (10, 'Cadano', 'ADA', 1.5, 50000000000.0, '2025-08-18 17:18:50'), (11, 'Solana', 'SOL', 150.0, 50000000000.0, '2025-08-18 17:18:50')]


In [44]:
# Get one coin
cursor.execute("SELECT*FROM coins WHERE symbol='BTC'")
bitcoin=cursor.fetchone() # Gets just the first result
print("Bitcoin data:", bitcoin)

Bitcoin data: (1, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:07:32')


In [45]:
# Get specific columns with a condition
cursor.execute("SELECT name,price FROM coins WHERE price >100")
expensive_coins=cursor.fetchall()
print("Expensive coins:", expensive_coins)

Expensive coins: [('Bitcoin', 51000.0), ('Bitcoin', 51000.0), ('Bitcoin', 51000.0), ('Bitcoin', 51000.0), ('Ethereum', 3000.0), ('Solana', 150.0), ('Bitcoin', 50000.0), ('Ethereum', 3000.0), ('Solana', 150.0)]


## Updating and Deleting

In [28]:
# Update bitcoin's price
cursor.execute("UPDATE coins SET price=51000 WHERE symbol='BTC'")
conn.commit()

In [46]:
# Delete cheap coins
cursor.execute("DELETE FROM coins WHERE price<21")
conn.commit()

#### Using Context Managers(best practice)
This automatically handles opening and closing the connection.

In [47]:
with sqlite3.connect(db_path) as conn:
    cursor=conn.cursor()
    cursor.execute("SELECT * FROM coins")
    print(cursor.fetchall())
    # No need to manually close, its handled by the with book, returns a list of turples(unstructured data)

[(1, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:07:32'), (2, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:19:48'), (3, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:21:13'), (4, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-18 13:22:28'), (5, 'Ethereum', 'ETH', 3000.0, 3500000000.0, '2025-08-18 13:22:28'), (7, 'Solana', 'SOL', 150.0, 50000000000.0, '2025-08-18 13:22:28'), (8, 'Bitcoin', 'BTC', 50000.0, 950000000000.0, '2025-08-18 17:18:50'), (9, 'Ethereum', 'ETH', 3000.0, 3500000000.0, '2025-08-18 17:18:50'), (11, 'Solana', 'SOL', 150.0, 50000000000.0, '2025-08-18 17:18:50')]


### Pandas and SQL integration
 Pandas makes working with SQL data even easier by connecting between data frames and SQL tables

 Reading SQL into Pandas.

In [48]:
import pandas as pd
with sqlite3.connect(db_path) as conn:
    #Read SQL query directly into a dataframe
    df=pd.read_sql("SELECT*FROM coins", conn)
    print("DataFrame from SQL")
    print(df.head())

DataFrame from SQL
   id      name symbol    price    markeT_cap         last_updated
0   1   Bitcoin    BTC  51000.0  9.500000e+11  2025-08-18 13:07:32
1   2   Bitcoin    BTC  51000.0  9.500000e+11  2025-08-18 13:19:48
2   3   Bitcoin    BTC  51000.0  9.500000e+11  2025-08-18 13:21:13
3   4   Bitcoin    BTC  51000.0  9.500000e+11  2025-08-18 13:22:28
4   5  Ethereum    ETH   3000.0  3.500000e+09  2025-08-18 13:22:28


### # Writing pandas to SQL

In [49]:
#create a new data frame
new_coins=pd.DataFrame({
    'name': ['Polkadot','Polygon' ],
    'symbol': ['DOT', 'MATIC'],
    'price': [7.5, 0.75],
    'market_cap': [7500000000,5000000000]
})

with sqlite3.connect(db_path) as conn:
# Write the DataFrame to a new SQL table
    new_coins.to_sql("new_coins", conn, index=False, if_exists='replace')
# verify that it works
df_new=pd.read_sql("SELECT * FROM new_coins", conn)
print("\nNew coin table:")
print(df_new)


New coin table:
       name symbol  price  market_cap
0  Polkadot    DOT   7.50  7500000000
1   Polygon  MATIC   0.75  5000000000


Nb: Include a with statement in all your cells to the data base

### Analyzing SQL Data with Pandas

In [52]:
with sqlite3.connect(db_path) as conn:
    df=pd.read_sql("SELECT * FROM coins", conn)
    # Now you can use all pandas operations
    print("n\Basic statistics:")
    print(df.describe())

    print("\nAverage price:", df['price']. mean())
    print("Total Market Cap:", df['markeT_cap']. sum())

n\Basic statistics:
              id         price    markeT_cap
count   9.000000      9.000000  9.000000e+00
mean    5.555556  28922.222222  5.396667e+11
std     3.395258  25965.338809  4.868731e+11
min     1.000000    150.000000  3.500000e+09
25%     3.000000   3000.000000  5.000000e+10
50%     5.000000  50000.000000  9.500000e+11
75%     8.000000  51000.000000  9.500000e+11
max    11.000000  51000.000000  9.500000e+11

Average price: 28922.222222222223
Total Market Cap: 4857000000000.0


In [53]:
# filter with pandas
expensive=df[df['price']>100]
print("\nExpensive coins:")
print(expensive[['name']])


Expensive coins:
       name
0   Bitcoin
1   Bitcoin
2   Bitcoin
3   Bitcoin
4  Ethereum
5    Solana
6   Bitcoin
7  Ethereum
8    Solana


# Joins and Relationships

#### Understanding table relationships
in real databases, data is often split across multiple related tables. For example coins table, coin_id, name, symbol
prices table: price_id, coin_id, price, date

#### Creating Related Tables

In [ ]:
with sqlite3.connect(db_path) as conn:
    cursor=conn.cursor()
    # Create coins table
    cursor.execute("DROP TABLE IF EXISTS coins")
    cursor.execute("""
                   CREATE TABLE coins(
                   coin_id INTEGER PRIMARY KEY AUTOINCREMENT,
                   name TEXT NOT NULL,
                   symbol TEXT NOT NULL UNIQUE)
                   """)
    
    # Create prices table with foreign key
    cursor.execute("""
                   CREATE TABLE IF NOT EXISTS prices(
                   price_id INTEGER PRIMARY KEY AUTOINCREMENT,
                   coin_id INTEGER  NOT NULL,
                   price REAL NOT NULL,
                   date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                   FOREIGN KEY(coin_id) REFERENCES coins(coin_id)
                   )
                  """ )
    
    # Insert some data
    cursor.execute("INSERT INTO coins(name,symbol) VALUES('Bitcoin', 'BTC')")
    cursor.execute("INSERT INTO coins(name,symbol) VALUES('Ethereum', 'ETH')")

    # Get the auto generated IDs
    eth_id=cursor.lastrowid # This would actually get you ETH's ID in real code; you would need to query for BTC's
    btc_id=eth_id-1
    # Insert price history
    cursor.executemany("""
    INSERT INTO prices(coin_id,price) 
    VALUES(?,?)""",
    [
        (btc_id, 50000),
        (btc_id, 51000),
        (eth_id, 3000),
        (eth_id, 3100)
    ])
    conn.commit()

### Performing Joins

#### Example of a Right Join

In [58]:
with sqlite3.connect(db_path) as conn:
    # Simple inner join
    df=pd.read_sql("""
    SELECT c.name, c.symbol, p.price, p.date
    FROM coins c
    INNER JOIN prices p ON c.coin_id=p.coin_id
    ORDER BY p.date DESC
    """, conn)
    # Coin_id to bridge the two tables
    print("\ncoin prices with join:")
    print(df)


coin prices with join:
       name symbol    price                 date
0   Bitcoin    BTC  50000.0  2025-08-19 13:15:40
1   Bitcoin    BTC  51000.0  2025-08-19 13:15:40
2  Ethereum    ETH   3000.0  2025-08-19 13:15:40
3  Ethereum    ETH   3100.0  2025-08-19 13:15:40


Inner join--> keeps only the rows that match in both tables and matches from the left table if available or present.

Left Join-->keeps all rows from the left table and matches from the right table if available or present.

Full Join-->keeps all rows from both tables matching where possible.

#### Example of a left Join
To get all coins even if they have no prices in the left table and matching anything familiar in all tables

In [60]:
with sqlite3.connect(db_path) as conn:
    df=pd.read_sql("""
    SELECT c.name, c.symbol,p.price,p.date
    FROM coins c                                    
    LEFT JOIN prices p ON c.coin_id=p.coin_id
    """, conn)
    print("\nAll coins with prices(if available):")
    print(df)


All coins with prices(if available):
       name symbol    price                 date
0   Bitcoin    BTC  50000.0  2025-08-19 13:15:40
1   Bitcoin    BTC  51000.0  2025-08-19 13:15:40
2  Ethereum    ETH   3000.0  2025-08-19 13:15:40
3  Ethereum    ETH   3100.0  2025-08-19 13:15:40


# Best Practices

In [61]:
# Use context managers--> tghe with statement
with sqlite3.connect(db_path) as conn:
    df=pd.read_sql("""
    SELECT * FROM coins c
    """, conn)

In [62]:
# Close connection with commit (if not using context manager)
conn=sqlite3.connect(db_path)
df=pd.read_sql("""
SELECT * FROM coins
""", conn)
conn.commit() # Saving the updates to database
conn.close() # Closing the database connection

In [63]:
# Index frequent columns to improve query performance for example you are frequently searching by 'symbol' or 'name' in the db
conn=sqlite3.connect(db_path)
cursor=conn.cursor()

In [65]:
# Create an index on name
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_coins_name ON coins(name);
""")
conn.commit()  # Because we didnt use a context manager.
conn.close()
print("Indexes created")

Indexes created


In [ ]:
with sqlite3.connect(db_path) as conn:
    df=pd.read_sql("""
    SELECT * FROM coins
    WHERE symbol='BTC'
    """, conn)
    print(df)

TypeError: read_sql() missing 1 required positional argument: 'con'